#  Green Screen #

This is a Green Screen for two inputs.



First you download the bit file.

In [ ]:
from pynq.overlays.video import *
from pynq.lib.video import *

base = VideoOverlay("video.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

Then start up the PRControl, the video will not work otherwise. It initailzes the Video Axi Switch so HDMI runs through the VDMA.

In [ ]:
from pynq.overlays.video import PRControl
pr_inst = PRControl()

The best video sources are computers where you can control the resoltuion.

In [ ]:
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode,PIXEL_BGR)
hdmi_out.start()
hdmi_in.start()
hdmi_in.tie(hdmi_out)

Here is a frame in VDMA.

In [ ]:
import PIL.Image
import cv2
frame = hdmi_in.readframe()
frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
image = PIL.Image.fromarray(frame)
image

The bitfile starts with the mux pr regions with two inputs(A,B) and a duplicated output (1,2) with the Green Screen filter preloaded.

Connect the HDMI_IN to input A (MUX_A) and VDMA to input B (MUX_B) and the first output (MUX_1) to HDMI_OUT

In [ ]:
pr_inst.connect("HDMI_IN","MUX_A")
pr_inst.connect("VDMA","MUX_B")
pr_inst.connect("MUX_1","HDMI_OUT")

The Mux has two outputs, but one is only enabled at the moment.  To enable the other output you need to do these commands:

In [ ]:
pr_inst.filter_cmd("MUX",7,1)
pr_inst.connect("MUX_2","VDMA")

Load in a background image

In [ ]:
import PIL.Image
import cv2
wid = hdmi_out.mode.width
hig = hdmi_out.mode.height
test = cv2.imread('background.jpg',1);

outframe = hdmi_out.newframe()
outframe[0:hig,0:wid] = test[0:hig,0:wid,:]

hdmi_out.writeframe(outframe)

In [ ]:
import PIL.Image
import cv2
frame = hdmi_in.readframe()
frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
image = PIL.Image.fromarray(frame)
image

We need to change the settings to allow the foreground to show
#### Green Screen Settings####
* 0 Set Red lower bound
* 1 Set Red upper bound
* 2 Set Green lower bound
* 3 Set Green upper bound
* 4 Set Blue lower bound
* 5 Set Blue upper bound
* 6 Set Input A as Foreground (Default is 1)
* 7 Use both outputs (Default is 0) This must be correct or the Axi-Stream pipeline may not work.

pr_inst.filter_cmd("filter name",cmd,data)

In [ ]:
pr_inst.filter_cmd("MUX",1,50)
pr_inst.filter_cmd("MUX",3,50)
pr_inst.filter_cmd("MUX",5,50)

In [ ]:
import PIL.Image
import cv2
frame = hdmi_in.readframe()
frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
image = PIL.Image.fromarray(frame)
image

Lets use a Widget to make this easier.

In [ ]:
import ipywidgets as widgets
from IPython.display import HTML, display

Red_IRS = widgets.IntRangeSlider(
    value=[0, 128],
    min=0,
    max=255,
    step=1,
    description='Valid Red:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

def update_red(*args):
    Red = (Red_IRS.value)
    pr_inst.filter_cmd("MUX",0,Red[0])
    pr_inst.filter_cmd("MUX",1,Red[1])

Red_IRS.observe(update_red, 'value')

Green_IRS = widgets.IntRangeSlider(
    value=[128, 255],
    min=0,
    max=255,
    step=1,
    description='Valid Green:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

def update_green(*args):
    Green = (Green_IRS.value)
    pr_inst.filter_cmd("MUX",2,Green[0])
    pr_inst.filter_cmd("MUX",3,Green[1])
    #print "green hi: %d, green lo: %d" % (Green[0], Green [1])
Green_IRS.observe(update_green, 'value')

Blue_IRS = widgets.IntRangeSlider(
    value=[0, 128],
    min=0,
    max=255,
    step=1,
    description='Valid Blue:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

def update_blue(*args):
    Blue = (Blue_IRS.value)
    pr_inst.filter_cmd("MUX",4,Blue[0])
    pr_inst.filter_cmd("MUX",5,Blue[1])
Blue_IRS.observe(update_blue, 'value')

widgets.VBox([Red_IRS, Green_IRS, Blue_IRS])

Save the Frame

In [ ]:
import PIL.Image
import cv2
frame = hdmi_in.readframe()
frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
image = PIL.Image.fromarray(frame)
image

In [ ]:
hdmi_out.close()
hdmi_in.close()